In [10]:
import numpy as np 
import pandas as pd  
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
import re

In [11]:
import re
import string

In [12]:
# pip install textblob

In [13]:
# pip install vaderSentiment

In [14]:
import textblob

In [15]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Read in and Prelim Stats

In [16]:
df = pd.read_csv('vaccine.csv')

In [17]:
df.shape

(6706, 13)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6706 entries, 0 to 6705
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   user_name         6706 non-null   object
 1   user_location     4487 non-null   object
 2   user_description  5706 non-null   object
 3   user_created      6706 non-null   object
 4   user_followers    6706 non-null   int64 
 5   user_friends      6706 non-null   int64 
 6   user_favourites   6706 non-null   int64 
 7   user_verified     6706 non-null   bool  
 8   date              6706 non-null   object
 9   text              6706 non-null   object
 10  hashtags          504 non-null    object
 11  source            6705 non-null   object
 12  is_retweet        6706 non-null   bool  
dtypes: bool(2), int64(3), object(8)
memory usage: 589.5+ KB


In [19]:
df.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,Kyle Sammin,Pennsylvania,"Lawyer, writer, Pennsylvanian. Sr. Contributor...",2010-04-30 13:27:01,1900,1658,62579,False,2020-09-14 23:59:59,"@timszostek Nah, this is just stupidity. Hate ...",NaN,Twitter for Android,False
1,RAM PRASAD CHOUDHARY,NaN,"The web of words is like a dense forest , be t...",2020-02-06 01:35:55,97,13,11985,False,2020-09-14 23:59:58,"""Pfizer CEO says drug company will know if COV...",NaN,Twitter for Android,False
2,Ahanefer,NaN,Knowledge seeker,2020-08-23 15:57:18,15,266,1114,False,2020-09-14 23:59:57,They made wearing anything on your face law. T...,NaN,Twitter Web App,False
3,sincerelyours,NaN,I am astounded at the level of emotional hosti...,2018-07-07 23:16:30,2729,4924,4013,False,2020-09-14 23:59:52,Quebec Premier skipped out of the stupidity va...,NaN,Twitter for iPhone,False
4,NATIONAL DIVORCE NOW!,NaN,🤡🌎,2016-03-04 00:57:07,3996,3716,130999,False,2020-09-14 23:59:50,@Regan0002112 @seungminkim Trump didn’t make t...,NaN,Twitter for iPhone,False


In [20]:
df.isna().sum()

user_name              0
user_location       2219
user_description    1000
user_created           0
user_followers         0
user_friends           0
user_favourites        0
user_verified          0
date                   0
text                   0
hashtags            6202
source                 1
is_retweet             0
dtype: int64

# Sentiment Vader

Thank you to Github user Amey23 for these functions! Brilliant stuff.

In [21]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)        
    return input_txt

def clean_tweets(tweets):
    # remove twitter Return handles (RT @xxx:)
    tweets = np.vectorize(remove_pattern)(tweets, "RT @[\w]*:") 
    
    # remove twitter handles (@xxx)
    tweets = np.vectorize(remove_pattern)(tweets, "@[\w]*")
    
    # remove URL links (httpxxx)
    tweets = np.vectorize(remove_pattern)(tweets, "https?://[A-Za-z0-9./]*")
    
    # remove special characters, numbers, punctuations (except for #)
    tweets = np.core.defchararray.replace(tweets, "[^a-zA-Z]", " ")
    
    return tweets

In [22]:
df['text'] = clean_tweets(df['text'])
df['text'].head()

0     Nah, this is just stupidity. Hate Trump if yo...
1    "Pfizer CEO says drug company will know if COV...
2    They made wearing anything on your face law. T...
3    Quebec Premier skipped out of the stupidity va...
4      Trump didn’t make the vaccine u science deny...
Name: text, dtype: object

In [23]:
df.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,Kyle Sammin,Pennsylvania,"Lawyer, writer, Pennsylvanian. Sr. Contributor...",2010-04-30 13:27:01,1900,1658,62579,False,2020-09-14 23:59:59,"Nah, this is just stupidity. Hate Trump if yo...",NaN,Twitter for Android,False
1,RAM PRASAD CHOUDHARY,NaN,"The web of words is like a dense forest , be t...",2020-02-06 01:35:55,97,13,11985,False,2020-09-14 23:59:58,"""Pfizer CEO says drug company will know if COV...",NaN,Twitter for Android,False
2,Ahanefer,NaN,Knowledge seeker,2020-08-23 15:57:18,15,266,1114,False,2020-09-14 23:59:57,They made wearing anything on your face law. T...,NaN,Twitter Web App,False
3,sincerelyours,NaN,I am astounded at the level of emotional hosti...,2018-07-07 23:16:30,2729,4924,4013,False,2020-09-14 23:59:52,Quebec Premier skipped out of the stupidity va...,NaN,Twitter for iPhone,False
4,NATIONAL DIVORCE NOW!,NaN,🤡🌎,2016-03-04 00:57:07,3996,3716,130999,False,2020-09-14 23:59:50,Trump didn’t make the vaccine u science deny...,NaN,Twitter for iPhone,False


In [24]:
scores = []

# Declare variables for scores
compound_list = []
positive_list = []
negative_list = []
neutral_list = []

for i in range(df['text'].shape[0]):
    #print(analyser.polarity_scores(sentiments_pd['text'][i]))
    compound = analyzer.polarity_scores(df['text'][i])["compound"]
    pos = analyzer.polarity_scores(df['text'][i])["pos"]
    neu = analyzer.polarity_scores(df['text'][i])["neu"]
    neg = analyzer.polarity_scores(df['text'][i])["neg"]
    
    scores.append({"Compound": compound,
                       "Positive": pos,
                       "Negative": neg,
                       "Neutral": neu
                  })

In [25]:
sentiments_score = pd.DataFrame.from_dict(scores)
df = df.join(sentiments_score)

In [26]:
df.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet,Compound,Positive,Negative,Neutral
0,Kyle Sammin,Pennsylvania,"Lawyer, writer, Pennsylvanian. Sr. Contributor...",2010-04-30 13:27:01,1900,1658,62579,False,2020-09-14 23:59:59,"Nah, this is just stupidity. Hate Trump if yo...",NaN,Twitter for Android,False,-0.5187,0.047,0.223,0.730
1,RAM PRASAD CHOUDHARY,NaN,"The web of words is like a dense forest , be t...",2020-02-06 01:35:55,97,13,11985,False,2020-09-14 23:59:58,"""Pfizer CEO says drug company will know if COV...",NaN,Twitter for Android,False,0.0000,0.000,0.000,1.000
2,Ahanefer,NaN,Knowledge seeker,2020-08-23 15:57:18,15,266,1114,False,2020-09-14 23:59:57,They made wearing anything on your face law. T...,NaN,Twitter Web App,False,-0.5267,0.000,0.159,0.841
3,sincerelyours,NaN,I am astounded at the level of emotional hosti...,2018-07-07 23:16:30,2729,4924,4013,False,2020-09-14 23:59:52,Quebec Premier skipped out of the stupidity va...,NaN,Twitter for iPhone,False,-0.4404,0.000,0.195,0.805
4,NATIONAL DIVORCE NOW!,NaN,🤡🌎,2016-03-04 00:57:07,3996,3716,130999,False,2020-09-14 23:59:50,Trump didn’t make the vaccine u science deny...,NaN,Twitter for iPhone,False,-0.5719,0.000,0.370,0.630


In [27]:
def sent_classifier(compound):
    if compound >= 0.05:
        return 'positive'
    if compound > -0.05 and compound < 0.05:
        return 'neutral'
    if compound <= -0.05:
        return 'negative'

In [28]:
df['sentiment'] = df.Compound

In [29]:
df.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet,Compound,Positive,Negative,Neutral,sentiment
0,Kyle Sammin,Pennsylvania,"Lawyer, writer, Pennsylvanian. Sr. Contributor...",2010-04-30 13:27:01,1900,1658,62579,False,2020-09-14 23:59:59,"Nah, this is just stupidity. Hate Trump if yo...",NaN,Twitter for Android,False,-0.5187,0.047,0.223,0.730,-0.5187
1,RAM PRASAD CHOUDHARY,NaN,"The web of words is like a dense forest , be t...",2020-02-06 01:35:55,97,13,11985,False,2020-09-14 23:59:58,"""Pfizer CEO says drug company will know if COV...",NaN,Twitter for Android,False,0.0000,0.000,0.000,1.000,0.0000
2,Ahanefer,NaN,Knowledge seeker,2020-08-23 15:57:18,15,266,1114,False,2020-09-14 23:59:57,They made wearing anything on your face law. T...,NaN,Twitter Web App,False,-0.5267,0.000,0.159,0.841,-0.5267
3,sincerelyours,NaN,I am astounded at the level of emotional hosti...,2018-07-07 23:16:30,2729,4924,4013,False,2020-09-14 23:59:52,Quebec Premier skipped out of the stupidity va...,NaN,Twitter for iPhone,False,-0.4404,0.000,0.195,0.805,-0.4404
4,NATIONAL DIVORCE NOW!,NaN,🤡🌎,2016-03-04 00:57:07,3996,3716,130999,False,2020-09-14 23:59:50,Trump didn’t make the vaccine u science deny...,NaN,Twitter for iPhone,False,-0.5719,0.000,0.370,0.630,-0.5719


In [30]:
df['sentiment'] = df.Compound.apply(sent_classifier)

In [31]:
df.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet,Compound,Positive,Negative,Neutral,sentiment
0,Kyle Sammin,Pennsylvania,"Lawyer, writer, Pennsylvanian. Sr. Contributor...",2010-04-30 13:27:01,1900,1658,62579,False,2020-09-14 23:59:59,"Nah, this is just stupidity. Hate Trump if yo...",NaN,Twitter for Android,False,-0.5187,0.047,0.223,0.730,negative
1,RAM PRASAD CHOUDHARY,NaN,"The web of words is like a dense forest , be t...",2020-02-06 01:35:55,97,13,11985,False,2020-09-14 23:59:58,"""Pfizer CEO says drug company will know if COV...",NaN,Twitter for Android,False,0.0000,0.000,0.000,1.000,neutral
2,Ahanefer,NaN,Knowledge seeker,2020-08-23 15:57:18,15,266,1114,False,2020-09-14 23:59:57,They made wearing anything on your face law. T...,NaN,Twitter Web App,False,-0.5267,0.000,0.159,0.841,negative
3,sincerelyours,NaN,I am astounded at the level of emotional hosti...,2018-07-07 23:16:30,2729,4924,4013,False,2020-09-14 23:59:52,Quebec Premier skipped out of the stupidity va...,NaN,Twitter for iPhone,False,-0.4404,0.000,0.195,0.805,negative
4,NATIONAL DIVORCE NOW!,NaN,🤡🌎,2016-03-04 00:57:07,3996,3716,130999,False,2020-09-14 23:59:50,Trump didn’t make the vaccine u science deny...,NaN,Twitter for iPhone,False,-0.5719,0.000,0.370,0.630,negative


# Unsupervised Learning

In [32]:
df['speech_clean'] = df['text']

#Remove numbers
alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
df['speech_clean'] = df['speech_clean'].apply(alphanumeric)

#Remove punctuation
punctuation = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x)
df['speech_clean'] = df['speech_clean'].apply(punctuation)

#Remove non-breaking space
xa0 = lambda x: re.sub('\xa0', ' ', x)
xad = lambda x: re.sub('\xad', ' ', x)
df['speech_clean'] = df['speech_clean'].apply(xa0).apply(xad)

#Remove dash
dash = lambda x: re.sub('–', ' ', x)
df['speech_clean'] = df['speech_clean'].apply(dash)

#Make lower case
lower = lambda x: x.lower()
df['speech_clean'] = df['speech_clean'].apply(lower)

#Replace hyphens
hyphen_1 = lambda x: re.sub('“', ' ', x)
hyphen_2 = lambda x: re.sub('„', ' ', x)
df['speech_clean'] = df['speech_clean'].apply(hyphen_1).apply(hyphen_2)

#Remove doubles space
spaces = lambda x: ' '.join(x.split())
df['speech_clean'] = df['speech_clean'].apply(spaces)

### Tokens, Lems and Stems

In [33]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from nltk.stem import WordNetLemmatizer 

In [34]:
stop = stopwords.words('english')

new_stop = (['abov', 'ani', 'becaus', 'befor', 'could', 
             'doe', 'dure', 'might', 'must', "n't", 'need', 'onc', 
             'onli', 'ourselv', 'sha', 'themselv', 'veri', 'whi', 'would', 'yourselv', 'covidvaccin',
            'vaccin', 'covid', 'coronavirus'])




stop = stop + new_stop

In [35]:
lemmatizer = WordNetLemmatizer() 
stemmer = SnowballStemmer('english')


In [36]:
def tokenize_and_stem(text):
    '''applies tokenization and stemming'''
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        token = stemmer.stem(token)
        if len(token) > 2:
            filtered_tokens.append(token)
    return filtered_tokens

In [37]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    '''shows topics identified in topic modeling'''
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

### Vectorize Me!

In [38]:
tfidf = TfidfVectorizer(stop_words=stop,tokenizer=tokenize_and_stem,min_df=.0025,max_df=.1)
sparse_tfidf = tfidf.fit_transform(df['speech_clean'])
tfidf_feature_names = tfidf.get_feature_names()

In [39]:
len(tfidf_feature_names)

557

In [40]:
nmf_10 = NMF(5,random_state=1)
nmf_10_topic = nmf_10.fit_transform(sparse_tfidf)

In [41]:
display_topics(nmf_10, tfidf_feature_names, 10, topic_names=None)



Topic  0
avail, everyon, world, biggest, say, produc, end, ceo, enough, year

Topic  1
get, flu, year, shot, peopl, like, even, one, virus, time

Topic  2
pfizer, know, work, octob, compani, drug, end, ceo, say, year

Topic  3
take, trump, peopl, rush, first, think, one, trust, want, approv

Topic  4
trial, astrazeneca, resum, hold, effect, serious, halt, oxford, least, phase


# Location Clean Up

In [42]:
df.user_location.value_counts().head(50)

United States               132
Washington, DC               93
London, England              70
USA                          55
New York, NY                 51
Los Angeles, CA              46
Canada                       40
Australia                    35
California, USA              31
New York, USA                30
Texas, USA                   29
Florida, USA                 28
UK                           25
Chicago, IL                  25
Boston, MA                   25
United Kingdom               24
Los Angeles                  22
San Francisco, CA            21
England, United Kingdom      20
Toronto, Ontario             20
Houston, TX                  20
London                       20
New York                     19
Raleigh, NC                  19
California                   18
New Jersey                   17
Washington, D.C.             17
Pennsylvania, USA            17
San Diego, CA                16
Ireland                      16
Philadelphia, PA             15
Earth   

In [43]:
def location_fixer(location):
    if location == '22.549709,88.401162':
        return "India"
    
    elif type(location) == str:
        if "Pakistan" in location:
            return "Pakistan"
        
        if "Ireland" in location:
            return "Ireland"
        
        if "India" in location:
            return "India"
        if "Ahmedabad" in location:
            return "India"
        if "Bhubaneswar" in location:
            return "India"
        if "Pune" in location:
            return "India"
        if "Guwahati" in location:
            return "India"
        if "Jammu" in location:
            return "India"
        if "Delhi" in location:
            return "India"
        if "india" in location:
            return "India"
        if 'Mumbai' in location:
            return 'India'
        if 'New Delhi' in location:
            return "India"
        if 'Bangalore' in location:
            return "India"
        if 'Hyderabad' in location:
            return "India"
        if 'Chennai' in location:
            return "India"
        if 'Kolkata' in location:
            return "India"
        if 'hyderabad' in location:
            return "India"
        if 'भारत' in location:
            return "India"
        if 'Chandigarh' in location:
            return "India"
        if 'BENGALURU' in location:
            return "India"
        if 'Lucknow' in location:
            return "India"
        if 'Kanpur' in location:
            return "India"
        
        
        if 'Thailand' in location:
            return "Thailand"
        
        if "Nepal" in location:
            return "Nepal"
        
        if 'UAE' in location:
            return "UAE"
        if 'Dubai' in location:
            return "UAE"
        

        
        if 'Quezon City' in location:
            return "Philippines"
        
        
        if 'Bangladesh' in location:
            return "Bangladesh"
       
        if 'USA' in location:
            return "United States"
        if 'NY' in location:
            return "United States"
        if 'MD' in location:
            return "United States"
        if 'DC' in location:
            return "United States"
        if 'TX' in location:
            return "United States"
        if 'CA' in location:
            return 'United States'
        if 'Atlanta' in location:
            return 'United States'
        if 'Los Angeles' in location:
            return 'United States'
        if 'IN' in location:
            return 'United States'
        if 'New York' in location:
            return 'United States'
        if 'SC' in location:
            return 'United States'
        if 'The North' in location:
            return 'United States'
        if 'MO' in location:
            return 'United States'
        if 'California' in location:
            return 'United States'
        if 'PA' in location:
            return "United States"
        if 'MA' in location:
            return "United States"
        if 'WA' in location:
            return "United States"
        if 'Massachusetts' in location:
            return "United States"
        if 'KS' in location:
            return "United States"
        if 'San Francisco' in location:
            return "United States"
        if 'Texas' in location:
            return "United States"
        if 'IL' in location:
            return "United States"
        if 'NC' in location:
            return "United States"
        if 'VA' in location:
            return "United States"
        if 'America' in location:
            return "United States"
        if 'AZ' in location:
            return "United States"
        if 'NV' in location:
            return "United States"
        if 'CO' in location:
            return "United States"
        if 'Florida' in location:
            return "United States"
        
        
        
        if 'Amsterdam' in location:
            return "The Netherlands"
        
        if 'Kenya' in location:
            return "Kenya"
    
        
        if 'Melbourne' in location:
            return "Australia"
        if 'Australia' in location:
            return "Australia"
        if 'Sydney' in location:
            return "Australia"
        if 'Brisbane' in location:
            return "Australia"
        
        if 'London' in location:
            return "United Kingdom"
        if 'England' in location:
            return "United Kingdom"
        if 'UK' in location:
            return "United Kingdom"
        if 'Manchester' in location:
            return "United Kingdom"
        
        if 'Canada' in location:
            return "Canada"
        if 'Toronto' in location:
            return "Canada"
        if 'Ontario' in location:
            return "Canada"
        
        
        if 'Belgium' in location:
            return "Belgium"
        
        if 'İstanbul' in location:
            return "Turkey"
        
        if 'World' in location:
            return "Turkey"

        
        
        else:
            return location
    else:
        return location

In [44]:
# df['user_location'] = df.user_location.apply(location_fixer)

In [45]:
df.head(45)

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet,Compound,Positive,Negative,Neutral,sentiment,speech_clean
0,Kyle Sammin,Pennsylvania,"Lawyer, writer, Pennsylvanian. Sr. Contributor...",2010-04-30 13:27:01,1900,1658,62579,False,2020-09-14 23:59:59,"Nah, this is just stupidity. Hate Trump if yo...",NaN,Twitter for Android,False,-0.5187,0.047,0.223,0.730,negative,nah this is just stupidity hate trump if you w...
1,RAM PRASAD CHOUDHARY,NaN,"The web of words is like a dense forest , be t...",2020-02-06 01:35:55,97,13,11985,False,2020-09-14 23:59:58,"""Pfizer CEO says drug company will know if COV...",NaN,Twitter for Android,False,0.0000,0.000,0.000,1.000,neutral,pfizer ceo says drug company will know if covi...
2,Ahanefer,NaN,Knowledge seeker,2020-08-23 15:57:18,15,266,1114,False,2020-09-14 23:59:57,They made wearing anything on your face law. T...,NaN,Twitter Web App,False,-0.5267,0.000,0.159,0.841,negative,they made wearing anything on your face law th...
3,sincerelyours,NaN,I am astounded at the level of emotional hosti...,2018-07-07 23:16:30,2729,4924,4013,False,2020-09-14 23:59:52,Quebec Premier skipped out of the stupidity va...,NaN,Twitter for iPhone,False,-0.4404,0.000,0.195,0.805,negative,quebec premier skipped out of the stupidity va...
4,NATIONAL DIVORCE NOW!,NaN,🤡🌎,2016-03-04 00:57:07,3996,3716,130999,False,2020-09-14 23:59:50,Trump didn’t make the vaccine u science deny...,NaN,Twitter for iPhone,False,-0.5719,0.000,0.370,0.630,negative,trump didn’t make the vaccine u science denyin...
5,Fear is the mind killer | #Dune2020,NAIROBI,Creative | Production and Product Designer | F...,2009-06-12 11:39:16,1589,1579,21597,False,2020-09-14 23:59:50,These people want to pay Russian Roulette with...,NaN,Twitter for Android,False,-0.0258,0.060,0.065,0.876,neutral,these people want to pay russian roulette with...
6,"Robert Key, II",NaN,NaN,2015-11-05 18:06:02,141,242,15902,False,2020-09-14 23:59:49,Go tell the mask manufacturers that. Dr. Seb...,NaN,Twitter for iPhone,False,0.6597,0.240,0.073,0.687,positive,go tell the mask manufacturers that dr sebi 😂 ...
7,swampwitch,some place in GA.,Southern momma and Nasty woman. Black lives m...,2020-02-06 14:23:46,387,737,19213,False,2020-09-14 23:59:45,Is the hope for a 2 shot and longevity like s...,NaN,Twitter for Android,False,0.6597,0.213,0.000,0.787,positive,is the hope for a shot and longevity like say ...
8,The Wear A Mask Chocobo,NaN,"I tweet about games, politics and football. Al...",2012-07-08 03:11:05,930,1062,29570,False,2020-09-14 23:59:45,soooooooooooo for those states saying they wil...,NaN,Twitter Web App,False,0.0000,0.000,0.000,1.000,neutral,soooooooooooo for those states saying they wil...
9,Kevin,NaN,Retired,2018-01-12 01:22:01,241,228,40627,False,2020-09-14 23:59:44,For those with a brain I would not trust the ...,NaN,Twitter for iPad,False,-0.1788,0.117,0.183,0.700,negative,for those with a brain i would not trust the t...


# Clean Up on Final Aisle

In [46]:
df_study = df[['user_name','user_location','date','text','Compound','sentiment']]

In [47]:
df_final = df[['user_name','user_location','date','text','sentiment']]

In [48]:
df_final.head()

,user_name,user_location,date,text,sentiment
0,Kyle Sammin,Pennsylvania,2020-09-14 23:59:59,"Nah, this is just stupidity. Hate Trump if yo...",negative
1,RAM PRASAD CHOUDHARY,NaN,2020-09-14 23:59:58,"""Pfizer CEO says drug company will know if COV...",neutral
2,Ahanefer,NaN,2020-09-14 23:59:57,They made wearing anything on your face law. T...,negative
3,sincerelyours,NaN,2020-09-14 23:59:52,Quebec Premier skipped out of the stupidity va...,negative
4,NATIONAL DIVORCE NOW!,NaN,2020-09-14 23:59:50,Trump didn’t make the vaccine u science deny...,negative


## Saving

In [49]:
# df.to_excel('nlp_json.xlsx')

In [50]:
df_final.to_excel('simple_vaccine_nlp.xlsx')

In [51]:
# df_study.to_excel('study_nlp.xlsx')